上传本地文件，并将文件内容存入向量数据库中。

In [ ]:
from pathlib import Path
from langchain_ollama import OllamaEmbeddings
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_core.documents import Document
from pymilvus import MilvusClient
import time

home_directory = str(Path.home())
file_path = f"{home_directory}/Documents/Data/order_202505_0.csv"

def load_csv() -> list:
    '''
    加载 CSV 为文档
    '''
    csvLoader = CSVLoader(
        file_path=file_path,
        encoding='utf-8',
        csv_args={
            "delimiter": ",", # 分隔符
            "quotechar": '"', # 引号字符：用于引用包含特殊字符的字段的单字符字符串，如定界符或者quotechar，或者包含换行符。它默认为'"'
            "fieldnames": ["订单ID", "用户ID", "下单时间", "支付时间", "支付金额", "设备类型", "支付方式", "产品ID", "产品名称", "自动续费", "会员类型"],
        },
        source_column="订单ID" # 为每一行创建的文档指定一个源，如果不制定，默认会将file_path作为source_column
    )

    docs = csvLoader.load()
    print(f"✅ 成功加载 {len(docs)} 条记录")
    return docs

def simple_embed(docs: list):
    '''
    简单向量化
    '''
    embeddings = OllamaEmbeddings(
        model="bge-m3",
        base_url = 'http://localhost:11434'
        # other params...
    )

    # 文本提取并生成向量
    texts = [doc.page_content for doc in docs]
    vectors = embeddings.embed_documents(texts)
    print(f"✅ 成功生成 {len(vectors)} 条向量，维度：{len(vectors[0])}")
    return vectors

def batch_embed(docs: list, batch_size=16, delay=0.2):
    '''
    分批向量化
    '''
    embeddings = OllamaEmbeddings(
        model="bge-m3",
        base_url = 'http://localhost:11434'
        # other params...
    )
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        print(f"🔄 嵌入第 {i} ~ {i + len(batch) - 1} 条")
        try:
            batch_vectors = embeddings.embed_documents(batch)
            all_embeddings.extend(batch_vectors)
        except Exception as e:
            print(f"❌ 嵌入失败: {e}")
        time.sleep(delay)  # 可选延迟，避免 Ollama 过载
    
    texts = [doc.page_content for doc in docs]
    vectors = batch_embed(texts, batch_size=16)
    print(f"✅ 嵌入完成，共 {len(vectors)} 条，维度：{len(vectors[0])}")
    return vectors

def save_vectors(docs: list, vectors):
    '''
    存储向量
    '''
    client = MilvusClient(
        uri="http://localhost:19530",
        token="root:Milvus",
        db_name="test_sport"
    )

    collection_name = "sport_order_vectors"
    dim = len(vectors[0])
    # 创建 Collection（如不存在）
    if collection_name not in client.list_collections():
        client.create_collection(
            collection_name=collection_name,
            dimension=dim,
            metric_type="COSINE"
        )
        print(f"📦 已创建 collection：{collection_name}")
    else:
        print(f"📦 已存在 collection：{collection_name}")
        
    # 插入向量 + metadata
    payload = []
    for i, doc in enumerate(docs):
        payload.append({
            "id": f"doc-{i}",  # 可用订单ID代替
            "vector": vectors[i],
            "metadata": {
                "source": doc.metadata.get("source", ""),
                "内容": doc.page_content
            }
        })

    client.insert(collection_name=collection_name, data=payload)
    print(f"🎉 向量插入成功，共插入：{len(payload)} 条")


def demo():
    docs = load_csv()
    voctors = batch_embed(docs)
    save_vectors(docs, voctors)